In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
frr

In [10]:
train_df=pd.read_csv(r'../playground-series-s5e5\train.csv',index_col=0)
val_df=pd.read_csv(r'../playground-series-s5e5\test.csv')
train_df.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
id,,,,,,,,
0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [11]:
#Split the data
x=train_df.drop('Calories',axis=1)
y=train_df['Calories']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=25)

In [12]:
#Convert the categorical columns to encodering
# X_train=pd.get_dummies(X_train,drop_first=True)
# X_test=test_df
# X_test=pd.get_dummies(X_test,drop_first=True)

#### 1. Linear Regression

In [13]:
#For linear regression
lr=LinearRegression()
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)

ValueError: could not convert string to float: 'female'

In [ ]:
#If Calories is negative give it 0
y_pred[y_pred<0]=0
y_pred[y_pred<0].sum()

np.float64(0.0)

In [ ]:
# Create the submission file
submit=pd.read_csv(r'../playground-series-s5e5\sample_submission.csv')
submit['Calories']=y_pred
submit.to_csv('submission.csv',index=False)

#### 2. CatBoost Model

In [ ]:
n_estimate=[10,50,100,200]
lr=np.linspace(0.001,1,10)
depth=[1,2,3,4]
scores=[]
for n in n_estimate:
    for r in lr:
        for d in depth:
            lgb=CatBoostRegressor(n_estimators=n,learning_rate=r,max_depth=d,random_state=25,cat_features=['Sex'],verbose=False)
            lgb.fit(x_train,y_train)
            y_pred=lgb.predict(x_test)
            y_pred[y_pred<0]=0
            scores.append([n,r,d,root_mean_squared_log_error(y_test,y_pred)])
scores_df=pd.DataFrame(scores,columns=['n_estimate','Learning rate','Depth','RMSLE'])
scores_df.sort_values(by='RMSLE',ascending=True,inplace=True)
scores_df.head()

KeyboardInterrupt: 

In [ ]:
best_model=CatBoostRegressor(n_estimators=200,learning_rate=0.556,max_depth=4,random_state=25,cat_features=['Sex'],verbose=False)
best_model.fit(x_train,y_train)
# y_pred =ada.predict(X_test)
best_y_pred=best_model.predict(val_df)
best_y_pred[best_y_pred<0]=0

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="male": Cannot convert 'male' to float

In [ ]:
submit=pd.read_csv(r'../playground-series-s5e5\sample_submission.csv')
submit['Calories']=best_y_pred
submit.to_csv('submission1.csv',index=False)

#### Using LabelEncoder on Sex column

In [ ]:
le=LabelEncoder()
x_train['Sex']=le.fit_transform(x_train['Sex'])
x_test['Sex']=le.transform(x_test['Sex'])

In [ ]:
x_train

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
id,,,,,,,
447304,0,22,150.0,50.0,22.0,98.0,40.4
29261,0,30,148.0,53.0,12.0,95.0,39.8
489234,1,21,187.0,90.0,8.0,92.0,39.9
224688,0,30,170.0,70.0,24.0,107.0,40.4
689365,0,65,179.0,74.0,25.0,103.0,40.8
...,...,...,...,...,...,...,...
130365,1,42,189.0,93.0,28.0,111.0,41.0
616591,1,70,191.0,93.0,24.0,93.0,40.9
6618,1,30,179.0,82.0,4.0,87.0,38.9


In [ ]:
val_df

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5
...,...,...,...,...,...,...,...,...
249995,999995,female,56,159.0,62.0,6.0,85.0,39.4
249996,999996,male,32,202.0,101.0,3.0,84.0,38.4
249997,999997,female,31,164.0,64.0,14.0,98.0,40.1
249998,999998,female,62,158.0,61.0,25.0,106.0,40.7


In [ ]:
best_model=CatBoostRegressor(n_estimators=200,learning_rate=0.556,max_depth=4,random_state=25,verbose=False)
best_model.fit(x_train,y_train)
# y_pred =ada.predict(X_test)
best_y_pred=best_model.predict(val_df)
best_y_pred[best_y_pred<0]=0

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="male": Cannot convert 'male' to float

In [ ]:
submit=pd.read_csv(r'../playground-series-s5e5\sample_submission.csv')
submit['Calories']=best_y_pred
submit.to_csv('submission.csv',index=False)